In [1]:
from fuzzywuzzy import fuzz
import os
from dotenv import load_dotenv
import streamlit as st
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from Levenshtein import distance as levenshtein_distance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
from streamlit.logger import get_logger
from metaphone import doublemetaphone


In [2]:
load_dotenv(verbose=True)

logger = get_logger(__name__)

In [3]:
def read_product_list(file_path):
    try:
        data = pd.read_csv(file_path)
        # print("product list", data)
        return data
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return None

In [5]:
def extract_product_name(question):
 
    client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") 
    )
    
    try:
        #response = openai.ChatCompletion.create(
        response = client.chat.completions.create(
            #model="gpt-3.5-turbo",
            model="gpt-4",
            messages=[
                {"role": "system", "content": "질문에서 제품명을 추출합니다."},
                {"role": "user", "content": question}
            ]
        )
        #product_name = response.choices[0].message['content'].strip()
        product_name = response.choices[0].message.content.strip()
        return product_name
    except Exception as e:
        print(f"OpenAI GPT-3 요청 중 오류가 발생했습니다: {e}")
        return None

In [6]:
def find_similar_products_sequence_matching(user_query, product_data):
    """
    Sequence Matching 알고리즘을 이용하여 가장 유사한 제품을 찾는 함수.
    """
    # 유사도 계산을 위한 함수 정의
    def calculate_similarity(product_name):
        # Sequence Matching 알고리즘을 사용하여 유사도 계산
        return SequenceMatcher(None, user_query, product_name).ratio()

    # 각 제품에 대해 유사도 계산
    product_data['Similarity'] = product_data['제품명'].apply(
        calculate_similarity)

    # 유사도에 따라 제품들을 정렬
    sorted_products = product_data.sort_values(
        by='Similarity', ascending=False)

    # 상위 3개 제품 반환
    #return sorted_products.head(3).to_string(index=False)
    return sorted_products.head(3)

In [7]:
# 제품 목록 읽기
#product_data = read_product_list("/Users/komasin4/Data/maum.ai/pulmuwon/prodlist_2.csv")
product_data = read_product_list("/Users/komasin4/Data/maum.ai/pulmuwon/mergelist_1.csv")

In [8]:
question = '제주 보리차는 얼마인가요?'


In [9]:
product_name = extract_product_name(question)
print(product_name)

제품명은 제주 보리차입니다.


In [10]:
similar_products = find_similar_products_sequence_matching(product_name, product_data)
print("find_similar_products_sequence_matching", similar_products)



find_similar_products_sequence_matching     구분  NO 대분류          제품명  Similarity
30  제품  31   -    THE순제주보리차    0.416667
5   제품   6   -    THE순제주보리차    0.416667
18  제품  19   -  우리쌀로 만든 오란다    0.230769


In [28]:
print(similar_products.__class__)

for index, row in similar_products.iterrows():
    #print(row)
    print(index, row['구분'], row['NO'], row.대분류, row.제품명, row.Similarity)
    
print(similar_products[['제품명']])

al = ','.join(map("'{}'".format, similar_products['제품명']))

print(al)

<class 'pandas.core.frame.DataFrame'>
30 제품 31 - THE순제주보리차 0.4166666666666667
5 제품 6 - THE순제주보리차 0.4166666666666667
18 제품 19 - 우리쌀로 만든 오란다 0.23076923076923078
            제품명
30    THE순제주보리차
5     THE순제주보리차
18  우리쌀로 만든 오란다
'THE순제주보리차','THE순제주보리차','우리쌀로 만든 오란다'
